# RAGFusion Module Usage Notebook

This notebook demonstrates how to use the `RAGFusion` module for Retrieval-Augmented Generation (RAG).

**Overview:**
- Load configuration parameters from a YAML file (`config.yaml`).
- Initialize the `RAGFusion` pipeline.
- Add documents (PDFs or DOCX files) into the vector database.
- Retrieve relevant document context based on a query.
- Generate a complete answer or stream an answer.

**Prerequisites:**
- Install required packages
- Ensure your document files (PDFs, DOCX) are available.

Below is an example `config.yaml` file:

```yaml
llm:
    model_id: "meta-llama/llama-4-scout-17b-16e-instruct"
    max_tokens: 500
    temperature: 0
    top_p: 0.1
    stop_sequences: []
vector_db:
    embedding_modelname: "intfloat/multilingual-e5-small"
    collection_name: "chroma_collection_rag_fusion"
    persist_directory: "./chroma_db"
chunking:
    chunk_size: 512
    chunk_overlap: 50
    separators: ["\n\n", " "]
file_reader:
    pdf:
        pdfloader: "PYMUPDF"
    docx:
        docxloader: "python-docx"
```

Also, ensure that you have set your GROQ credentials as environment variables.

If you have the environment variables setup in .env file, you can load them using load_dotenv

In [ ]:
import sys
import os
from dotenv import load_dotenv
load_dotenv()

Optionally, set your GROQ credentials here if they are not already set as environment variables.

In [ ]:
# os.environ["GROQ_APIKEY"] = "<your-groq-api-key>"

In [3]:
# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

### Load configuration from the YAML file.

In [ ]:
import yaml

with open("../rag_pipelines/rag_fusion/config.yaml", "r") as f:
    config = yaml.safe_load(f)

print("Configuration loaded:")
config

## Initialize the RAGFusion Pipeline

The RAGFusion module uses the configuration to set up the language model and vector database.

In [5]:
from rag_pipelines.rag_fusion import RAGFusion

In [ ]:
# Initialize the pipeline with the configuration dictionary.
rag_pipeline = RAGFusion(config)
print("RAGFusion pipeline initialized!")

## Add Documents

Use the `add_documents` method to load and process documents into the vector database.

Provide a list of file paths (e.g., PDFs or DOCX files).

In [ ]:
document_files = ["../data/wxA Actions and basic IAM.PDF", "../data/wxA Custom Extensions.PDF"]

rag_pipeline.add_documents(document_files)
print("Documents added successfully to the vector database.")

## Retrieve Relevant Context and Generate Answer using get_answer

In this cell, we first retrieve and fuse relevant documents for the given query by generating related queries and applying reciprocal rank fusion using the `get_context` method.

Then we use these documents together with the query to generate an answer via the `get_answer` method.


In [ ]:
query = "What are custom extensions?"
relevant_docs = rag_pipeline.get_context(query, k=4)
relevant_docs

In [ ]:
# Generate an answer using get_answer with the query and retrieved documents.
answer_response = rag_pipeline.get_answer(query, relevant_docs)
print("Generated Answer:")
print(answer_response.content)

## Generate an Answer with a Single Method Call using respond_to_query

Use the `respond_to_query` method to generate a complete answer using the retrieved documents.
This method returns both the generated answer and the list of documents used.

The `respond_to_query` method combines both the retrieval of relevant context and answer generation into one step.
This means you don't have to call `get_context` and `get_answer` separately.

In [ ]:
answer, relevant_docs = rag_pipeline.respond_to_query(query, k=4)
print("Generated Answer:")
print(answer.content)

## Stream Answer

Alternatively, you can stream the answer token-by-token using the `stream_answer` method.
This example iterates over token chunks to get the full response.

In [ ]:
print("Streaming answer:")
stream_iterator = rag_pipeline.stream_answer(query, k=4)
for token_chunk in stream_iterator:
    print(token_chunk.content, end="")

# End of RAGFusion Module Usage Notebook

In this notebook, we demonstrated:
- Reading configuration from a YAML file.
- Initializing the RAGFusion pipeline.
- Adding documents to the vector database.
- Retrieving relevant context for a query.
- Generating a complete answer and streaming the answer.

Adjust file paths, query text, and configuration parameters as needed.